## Getting Mountain Information from Summit Post

This notebook scrapes the name, location, and the cover image URL for each mountain from the top N pages of mountains, sorted by descending number of hits from https://summitpost.org, a crowd sourced resource for mountaineering and hiking information.

In [1]:
import requests
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import cssutils


First, I get all the unique URLs for each mountain.  Each page lists 24 mountains.

In [2]:
mtn_urls = []

def mtns_top_hits(num_pages):
    for i in range(1,num_pages+1):
        top_url = f'https://www.summitpost.org/mountain/rock/?object_type=1&search_select_1=name_only&contributor_id=&order_type_1=DESC&object_name_1=&sort_select_1=hits&page={i}'
        response = requests.get(top_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "lxml")
            data = soup.find_all('div',attrs={'class':'item-data'})
            for div in data:
                links = div.find_all('a')
                for a in links[::2]: # pulling every other link because every other link is a "parent"
                    mtn_urls.append("http://www.summitpost.org" + a['href'])
        else:
            print(f'Response code error: {response.status_code}')
    return mtn_urls

In [3]:
# get a list of 960 mountains from the top 40 pages from the site
mtns_top_hits(40)

['http://www.summitpost.org/mount-whitney/150227',
 'http://www.summitpost.org/mount-rainier/150291',
 'http://www.summitpost.org/mount-shasta/150188',
 'http://www.summitpost.org/mount-hood/150189',
 'http://www.summitpost.org/denali/150199',
 'http://www.summitpost.org/mount-elbert/150325',
 'http://www.summitpost.org/katahdin/150219',
 'http://www.summitpost.org/aconcagua/150197',
 'http://www.summitpost.org/mount-adams/150198',
 'http://www.summitpost.org/grand-teton/150312',
 'http://www.summitpost.org/longs-peak/150310',
 'http://www.summitpost.org/matterhorn-monte-cervino/150235',
 'http://www.summitpost.org/mont-blanc/150245',
 'http://www.summitpost.org/eiger/150228',
 'http://www.summitpost.org/mount-mansfield/150938',
 'http://www.summitpost.org/humphreys-peak/150241',
 'http://www.summitpost.org/hatu-peak/154227',
 'http://www.summitpost.org/wheeler-peak-nm/150429',
 'http://www.summitpost.org/mt-timpanogos-ut/151365',
 'http://www.summitpost.org/mount-baker/150195',
 'http

Now that I have all the URLs, I go to each URL and get the relevant information.

In [4]:
# test URL to get location data

response = requests.get('https://www.summitpost.org/mont-blanc/150245')
soup = BeautifulSoup(response.text, "lxml")
        
test_loc = soup.find('div',attrs={'class':'location'}).find('span').text

test_loc

'Haute-Savoie/Aosta (Mont Blanc), France/Italy, Europe'

In [5]:
# test URL to get cover image URL

response = requests.get('https://www.summitpost.org/y-mountain/224762')
soup = BeautifulSoup(response.text, "lxml")
        
div_style = soup.find('div',attrs={'class':'cover-image'})['style']

style = cssutils.parseStyle(div_style)

test_imgurl = style['background-image']
test_imgurl = test_imgurl.replace('url(', '').replace(')', '')
test_imgurl

'https://sp-images.summitpost.org/224771.jpg?auto=format&fit=max&h=800&ixlib=php-2.1.1&q=35&s=0d8c9513b6f1d46e331b5b7025a2c7d7'

In [6]:
def get_mtn_info(urls):
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")
        
        # get location, if there's an issue, append the URL so you can manually fix later
        try:
            loc = soup.find('div',attrs={'class':'location'}).find('span').text
            locations.append(loc)
        except:
            locations.append(url)
            
        # get mountain name, if there's an issue, append the URL so you can manually fix later
        try:
            mountain = soup.find('div',attrs={'class':'content-title'}).text
            mountains.append(mountain)
        except:
            mountains.append(url)
            
        # get cover image URL, if there's an issue, append the URL so you can manually fix later
        try:
            div_style = soup.find('div',attrs={'class':'cover-image'})['style']

            style = cssutils.parseStyle(div_style)

            img_url = style['background-image']
            img_url = img_url.replace('url(', '').replace(')', '')
            img_urls.append(img_url)
        except:
            img_urls.append(url)
                
        time.sleep(.5+2*random.random())

In [7]:
mountains = []
locations = []
img_urls = []

get_mtn_info(mtn_urls)
print(mountains[:10])
print(locations[:10])
print(img_urls[:10])

['Mount Whitney', 'Mount Rainier', 'Mount Shasta', 'Mount Hood', 'Denali', 'Mount Elbert', 'Katahdin', 'Aconcagua', 'Mount Adams', 'Grand Teton']
['California, United States, North America', 'Washington, United States, North America', 'California, United States, North America', 'Oregon, United States, North America', 'Alaska, United States, North America', 'Colorado, United States, North America', 'Maine, United States, North America', 'Mendoza, Argentina, South America', 'Washington, United States, North America', 'Wyoming, United States, North America']
['https://sp-images.summitpost.org/469726.JPG?auto=format&fit=max&h=800&ixlib=php-2.1.1&q=35&s=7225b40bb8b15a1e3df43067fc820375', 'https://sp-images.summitpost.org/457178.jpg?auto=format&fit=max&h=800&ixlib=php-2.1.1&q=35&s=2dfab48d42c4250081216e0e7b8c6859', 'https://sp-images.summitpost.org/878331.jpg?auto=format&fit=max&h=800&ixlib=php-2.1.1&q=35&s=08e7844750df801767744669e8332445', 'https://sp-images.summitpost.org/5175.jpg?auto=fo

In [91]:
df_mtn_loc = pd.DataFrame({'mountain': mountains, 'location': locations, 'img_url': img_urls})
print(df_mtn_loc.info())
df_mtn_loc.sample(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 3 columns):
mountain    960 non-null object
location    960 non-null object
img_url     960 non-null object
dtypes: object(3)
memory usage: 22.6+ KB
None


,mountain,location,img_url
577,Lizard Head,"Colorado, United States, North America",https://sp-images.summitpost.org/647158.jpg?au...
242,Sierra Buttes,"California, United States, North America",https://sp-images.summitpost.org/351573.JPG?au...
348,Vestal Peak,"Colorado, United States, North America",https://sp-images.summitpost.org/92536.jpg?aut...
392,High Rock,"Maryland, United States, North America",https://sp-images.summitpost.org/313250.JPG?au...
445,Standing Indian Mountain,"North Carolina, United States, North America",https://sp-images.summitpost.org/88707.JPG?aut...
543,Mont Saint-Hilaire,"Quebec, Canada, North America",https://sp-images.summitpost.org/48204.jpg?aut...
9,Grand Teton,"Wyoming, United States, North America",https://sp-images.summitpost.org/844255.jpg?au...
837,Schwarzhorn (Grindelwald),"Bern, Switzerland, Europe",https://sp-images.summitpost.org/464113.jpg?au...
158,Gothics,"New York, United States, North America",https://sp-images.summitpost.org/31161.JPG?aut...
795,Suicide Rock,"California, United States, North America",https://sp-images.summitpost.org/39599.jpg?aut...


In [92]:
pd.options.display.max_rows = 999
df_mtn_loc

,mountain,location,img_url
0,Mount Whitney,"California, United States, North America",https://sp-images.summitpost.org/469726.JPG?au...
1,Mount Rainier,"Washington, United States, North America",https://sp-images.summitpost.org/457178.jpg?au...
2,Mount Shasta,"California, United States, North America",https://sp-images.summitpost.org/878331.jpg?au...
3,Mount Hood,"Oregon, United States, North America",https://sp-images.summitpost.org/5175.jpg?auto...
4,Denali,"Alaska, United States, North America",https://sp-images.summitpost.org/831080.JPG?au...
5,Mount Elbert,"Colorado, United States, North America",https://sp-images.summitpost.org/844972.JPG?au...
6,Katahdin,"Maine, United States, North America",https://sp-images.summitpost.org/74769.jpg?aut...
7,Aconcagua,"Mendoza, Argentina, South America",https://sp-images.summitpost.org/146685.jpg?au...
8,Mount Adams,"Washington, United States, North America",https://sp-images.summitpost.org/80003.jpg?aut...
9,Grand Teton,"Wyoming, United States, North America",https://sp-images.summitpost.org/844255.jpg?au...


In [93]:
df_mtn_loc.iloc[391][1]

'Irian Jaya (Or Papua), Indonesia, Australia/Oceana'

Scrolling through the dataframe, there are a few mountains where the information didn't pull in correctly.  Let's go through and manually correct these.

In [94]:
df_mtn_loc.iloc[70][1] = 'Hawaii, United States, Oceana'

df_mtn_loc.iloc[143][1] = 'New South Wales, Australia, Oceana'

df_mtn_loc.iloc[147][1] = 'Tibet/Khumbu, China/Nepal, Asia'

df_mtn_loc.iloc[181][1] = 'Hawaii, United States, Oceana'

df_mtn_loc.iloc[184][0] = 'Mount Tyndall'
df_mtn_loc.iloc[184][1] = 'California, United States, North America'

df_mtn_loc.iloc[186][1] = 'Canterbury, New Zealand, Oceana'

df_mtn_loc.iloc[225][1] = 'Italy/Switzerland, Europe'

df_mtn_loc.iloc[278][1] = 'Nepal, Asia'

df_mtn_loc.iloc[306][1] = 'Pangnirtung Baffin Island/Nunavut, Canada, North America'

df_mtn_loc.iloc[318][1] = 'Washington, United States, North America'

df_mtn_loc.iloc[391][1] = 'Irian Jaya (Or Papua), Indonesia, Oceana'

df_mtn_loc.iloc[473][1] = 'Wyoming, United States, North America'

df_mtn_loc.iloc[483][1] = 'Hawaii, United States, Oceana'

df_mtn_loc.iloc[497][1] = 'Westland, New Zealand, Oceana'

df_mtn_loc.iloc[587][1] = 'Pangnirtung Baffin Island/Nunavut, Canada, North America'

df_mtn_loc.iloc[641][1] = 'China/Nepal, Asia'

df_mtn_loc.iloc[747][0] = 'Mount Lola'
df_mtn_loc.iloc[747][1] = 'California, United States, North America'

df_mtn_loc.iloc[886][1] = 'Agat, Guam, Oceania'

df_mtn_loc.iloc[909][1] = 'Alberta, Canada, North America'

In [95]:
df_mtn_loc

,mountain,location,img_url
0,Mount Whitney,"California, United States, North America",https://sp-images.summitpost.org/469726.JPG?au...
1,Mount Rainier,"Washington, United States, North America",https://sp-images.summitpost.org/457178.jpg?au...
2,Mount Shasta,"California, United States, North America",https://sp-images.summitpost.org/878331.jpg?au...
3,Mount Hood,"Oregon, United States, North America",https://sp-images.summitpost.org/5175.jpg?auto...
4,Denali,"Alaska, United States, North America",https://sp-images.summitpost.org/831080.JPG?au...
5,Mount Elbert,"Colorado, United States, North America",https://sp-images.summitpost.org/844972.JPG?au...
6,Katahdin,"Maine, United States, North America",https://sp-images.summitpost.org/74769.jpg?aut...
7,Aconcagua,"Mendoza, Argentina, South America",https://sp-images.summitpost.org/146685.jpg?au...
8,Mount Adams,"Washington, United States, North America",https://sp-images.summitpost.org/80003.jpg?aut...
9,Grand Teton,"Wyoming, United States, North America",https://sp-images.summitpost.org/844255.jpg?au...


In [96]:
# split location into a temporary location and continent column

df_mtn_loc[['temp_loc','continent']] = df_mtn_loc['location'].str.rsplit(',', n=1, expand=True)
df_mtn_loc.drop(['location'], axis=1, inplace=True)
df_mtn_loc

,mountain,img_url,temp_loc,continent
0,Mount Whitney,https://sp-images.summitpost.org/469726.JPG?au...,"California, United States",North America
1,Mount Rainier,https://sp-images.summitpost.org/457178.jpg?au...,"Washington, United States",North America
2,Mount Shasta,https://sp-images.summitpost.org/878331.jpg?au...,"California, United States",North America
3,Mount Hood,https://sp-images.summitpost.org/5175.jpg?auto...,"Oregon, United States",North America
4,Denali,https://sp-images.summitpost.org/831080.JPG?au...,"Alaska, United States",North America
5,Mount Elbert,https://sp-images.summitpost.org/844972.JPG?au...,"Colorado, United States",North America
6,Katahdin,https://sp-images.summitpost.org/74769.jpg?aut...,"Maine, United States",North America
7,Aconcagua,https://sp-images.summitpost.org/146685.jpg?au...,"Mendoza, Argentina",South America
8,Mount Adams,https://sp-images.summitpost.org/80003.jpg?aut...,"Washington, United States",North America
9,Grand Teton,https://sp-images.summitpost.org/844255.jpg?au...,"Wyoming, United States",North America


In [113]:
# split temporary location into location and country columns

#df_mtn_loc[['location','country']] = df_mtn_loc['temp_loc'].str.split(',', n=1, expand=True)
#df_mtn_loc.drop(['temp_loc'], axis=1, inplace=True)
#df_mtn_loc

location = []
country = []

for i in range(0,len(df_mtn_loc)):
    if ',' in str(df_mtn_loc.temp_loc[i]):
        location.append(df_mtn_loc.temp_loc[i].rsplit(', ')[0])
        country.append(df_mtn_loc.temp_loc[i].rsplit(', ')[1])
    else:
        location.append(None)
        country.append(df_mtn_loc.temp_loc[i])

In [117]:
df_mtn_loc['location'] = location
df_mtn_loc['country'] = country

df_mtn_loc

,mountain,img_url,temp_loc,continent,location,country
0,Mount Whitney,https://sp-images.summitpost.org/469726.JPG?au...,"California, United States",North America,California,United States
1,Mount Rainier,https://sp-images.summitpost.org/457178.jpg?au...,"Washington, United States",North America,Washington,United States
2,Mount Shasta,https://sp-images.summitpost.org/878331.jpg?au...,"California, United States",North America,California,United States
3,Mount Hood,https://sp-images.summitpost.org/5175.jpg?auto...,"Oregon, United States",North America,Oregon,United States
4,Denali,https://sp-images.summitpost.org/831080.JPG?au...,"Alaska, United States",North America,Alaska,United States
5,Mount Elbert,https://sp-images.summitpost.org/844972.JPG?au...,"Colorado, United States",North America,Colorado,United States
6,Katahdin,https://sp-images.summitpost.org/74769.jpg?aut...,"Maine, United States",North America,Maine,United States
7,Aconcagua,https://sp-images.summitpost.org/146685.jpg?au...,"Mendoza, Argentina",South America,Mendoza,Argentina
8,Mount Adams,https://sp-images.summitpost.org/80003.jpg?aut...,"Washington, United States",North America,Washington,United States
9,Grand Teton,https://sp-images.summitpost.org/844255.jpg?au...,"Wyoming, United States",North America,Wyoming,United States


In [118]:
# reorder columns

df_mtn_loc = df_mtn_loc[['mountain','continent','country','location','img_url']]
df_mtn_loc

,mountain,continent,country,location,img_url
0,Mount Whitney,North America,United States,California,https://sp-images.summitpost.org/469726.JPG?au...
1,Mount Rainier,North America,United States,Washington,https://sp-images.summitpost.org/457178.jpg?au...
2,Mount Shasta,North America,United States,California,https://sp-images.summitpost.org/878331.jpg?au...
3,Mount Hood,North America,United States,Oregon,https://sp-images.summitpost.org/5175.jpg?auto...
4,Denali,North America,United States,Alaska,https://sp-images.summitpost.org/831080.JPG?au...
5,Mount Elbert,North America,United States,Colorado,https://sp-images.summitpost.org/844972.JPG?au...
6,Katahdin,North America,United States,Maine,https://sp-images.summitpost.org/74769.jpg?aut...
7,Aconcagua,South America,Argentina,Mendoza,https://sp-images.summitpost.org/146685.jpg?au...
8,Mount Adams,North America,United States,Washington,https://sp-images.summitpost.org/80003.jpg?aut...
9,Grand Teton,North America,United States,Wyoming,https://sp-images.summitpost.org/844255.jpg?au...


In [119]:
# exporting cleaned data to CSV

df_mtn_loc.to_csv('./mtn_locations.csv', index=False)